In [ ]:
import pandas as pd
import numpy as np
import torch
import os
import gc


In [ ]:
import pickle

In [ ]:
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

In [ ]:
import cv2

In [ ]:

import torchvision.models.detection as tmd
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor as FRP
import inspect
import torchvision
#converting the customised rcnn model
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor


In [ ]:
root_dir = 'D:\\Drone-Object-Detection\\VisDroneDataset\\'
changed_dir = 'D:\\Drone-Object-Detection\\VisDroneDataset_changed\\'
test_dir = 'test\\'
train_dir = 'train\\'
val_dir = 'validation\\'
labels_map = ['Background', 'Pedestrian', 'People', 'Bycicle', 'Car', 'Van', 'Truck',
                'Tricycle', 'Awning-tricycle', 'Bus', 'Motor', 'Others']

In [ ]:
def normalize_hist(channel_array):
    def Sh(h,i):
        return sum(h[:i])
    h = np.bincount(channel_array.flatten())
    H = sum(h)
    h = h/H
    
    for i in range(channel_array.shape[0]):
        for j in range(channel_array.shape[1]):
            channel_array[i][j] = 255 * Sh(h,channel_array[i][j])
                                         
    return channel_array

In [ ]:
def color_histogram_equalization(image):
    
    b,g,r = image[0].astype(np.uint8),image[1].astype(np.uint8),image[2].astype(np.uint8)
    
    b_eq = normalize_hist(b)
    g_eq = normalize_hist(g)
    r_eq = normalize_hist(r)
    
    equalized_image = cv2.merge((b_eq, g_eq, r_eq))
    #equalized_image = np.stack((b_eq, g_eq, r_eq))
    return equalized_image



In [ ]:

def hsv_histogram_equalization(image):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv_image)
    v_eq = cv2.equalizeHist(v)
    equalized_hsv = cv2.merge((h, s, v_eq))
    equalized_image = cv2.cvtColor(equalized_hsv, cv2.COLOR_HSV2BGR)
    
    return equalized_image

In [ ]:
def no_croppping(to_save,idx, image_path,bounding_boxes):
    to_file=''
    for index, box in enumerate(bounding_boxes): 
        x1_b, y1_b,width , height, a, b, c,d = box
        to_file+=f"{x1_b},{y1_b},{width},{height},{a},{b},{c},{d}\n"


    opencvImage = cv2.imread(image_path)
    cv2.imwrite(f'{to_save}train\\smart_images\\output_image_not_changed_{idx}.png',opencvImage)
    with open(f'{to_save}train\\smart_annotations\\bounding_boxes_not_changed_{idx}.txt', 'w') as bbox_file:
                bbox_file.write(to_file)

In [ ]:
from PIL import Image

def crop_and_save_boxes(to_save, image_path, txt_path,idx,img_shape=(480,480)):
    # Load the image
    original_image = Image.open(image_path)   
    bounding_boxes=[]
    with open(txt_path, 'r') as file:#check whether 0.8 are small
        bounding_boxes = np.array([list(map(int, line.split(','))) for line in file.readlines()])
        
        
        
    
    w_new = img_shape[0]
    h_new = img_shape[1]
    w,h = original_image.size
    # Calculate the number of 480x480 boxes to create
    num_boxes_horizontal = w// w_new  # Ceiling division
    num_boxes_vertical = h // h_new

    # Iterate over boxes and crop the image
    for i in range(num_boxes_horizontal):
        for j in range(num_boxes_vertical):
            # Calculate box coordinates
            x1_new = i * w_new
            y1_new =  j * h_new
            x2_new = (i+1) * w_new
            y2_new = (j+1) * h_new
            

            # Crop the image
            cropped_image = original_image.crop((x1_new, y1_new, x2_new, y2_new))

            # Save the cropped image
            #cropped_image.save(f'{to_save}train\\images_cropped\\output_image_{idx}_{i}_{j}.png')
            opencvImage = cv2.cvtColor(np.array(cropped_image), cv2.COLOR_RGB2BGR)
            
            to_file=''
            bb_new = []
            obs_num=0
            needed_el = 0
            for index, box in enumerate(bounding_boxes):
                
                x1_b, y1_b,width , height, a, b, c,d = box
                x2_b = x1_b + width
                y2_b = y1_b + height


                if (x1_b > x2_new or y1_b > y2_new) or (x2_b < x1_new or y2_b < y1_new): # no itersection

                    continue
                else:
                    
                    x1_b_new = max(0, x1_b - x1_new)
                    y1_b_new = max(0, y1_b - y1_new)
                    x2_b_new = min(x2_new - x1_new, x2_b - x1_new)
                    y2_b_new = min(y2_new - y1_new, y2_b - y1_new)#bug here
                    
                    to_file+=f"{x1_b_new},{y1_b_new},{x2_b_new - x1_b_new},{y2_b_new - y1_b_new},{a},{b},{c},{d}\n"
                    obs_num+=1
                    #needed_el += (b==6 or b==7 or b==8 or b==9)
                    if(x1_b_new < 0 or y1_b_new < 0 or  x2_b_new > 480 or y2_b_new > 480):
                        print(to_file,txt_path)
                        
                    
                #labels = bb_new[:,5]
                #small = [i  for i in bb_new if ]


            if(obs_num < 10) :
                continue

            with open(f'{to_save}train\\smart_annotations\\bounding_boxes_{idx}_{i}_{j}.txt', 'w') as bbox_file:
                bbox_file.write(to_file)
            cv2.imwrite(f'{to_save}train\\smart_images\\output_image_{idx}_{i}_{j}.png', opencvImage)


            

# Example usage
image_path = 'path/to/your/image.jpg'
txt_path = 'path/to/your/bounding_boxes.txt'


In [ ]:
def prepare_images(from_directory,to_directory, shape):
    images = os.listdir(from_directory+'images\\')
    annotations = os.listdir(from_directory+'annotations\\')
   
    for idx,(img_dir,ann_dir) in enumerate(zip(images, annotations)):
        
        image = cv2.imread(from_directory+'images\\'+ img_dir)  
        
        #image = sharp_img(image)
        
        img_width = image.shape[1]
        img_height = image.shape[0]
        
        new_width = shape[0]
        new_height = shape[1]
        if shape == (-1,-1):
            image = cv2.resize(image,(shape[0],shape[1]),interpolation = cv2.INTER_AREA)
        image = hsv_histogram_equalization(image)
        boxes = []
        labels = []
        with open(from_directory+'annotations\\'+ann_dir,'r') as f_f,open(to_directory+'annotations\\'+ann_dir,'w') as f_t:
            new_f = ''
            for line in f_f:
                boxes = []
                data = line.split(',')#strip
                #print(list(map(float, data[:5])))
                box_left, box_top, box_width, box_height,smth1, category,smth2,smth3 = map(float, data)
               
                box_width = 1 if box_width < 1 else box_width
                box_height = 1 if box_height < 1 else box_height
                
                box_right = box_left + box_width
                
                box_bottom = box_top + box_height
                #rescale box            
                box_left = (box_left/img_width) * new_width
                box_right = (box_right/img_width) * new_width
                box_top = (box_top/img_height) * new_height
                box_bottom = (box_bottom/img_height) * new_height
                boxes += [box_left,box_top,box_right, box_bottom]
                #print(boxes)
                new_f += ''.join([str(int(c))+',' for c in boxes])
                new_f+=str(int(smth1))+','
                new_f += str(int(category))+','
                new_f += str(int(smth2))+','
                new_f += str(int(smth3))+'\n'
                
                
            f_t.write(new_f)
            
        
        if(idx % 100==0):
            print(f'imf N {idx} written')
        #print('img written')
        cv2.imwrite(to_directory+'images\\'+ img_dir,image)
        

In [ ]:
#prepare_images(root_dir+train_dir,changed_dir+train_dir ,(480,480))
def crop():
    images = os.listdir(root_dir+train_dir+'images\\')
    annotations = os.listdir(root_dir+train_dir+'annotations\\')

    for idx,(img_dir,ann_dir) in enumerate(zip(images, annotations)):
        crop_and_save_boxes(changed_dir, root_dir+train_dir+'images\\'+img_dir,root_dir+train_dir+'annotations\\'+ann_dir,\
                            idx,img_shape = (480,480))
        if(idx % 100 == 0):
            print(idx)

In [ ]:
crop()

In [ ]:
class VisDroneDataset(Dataset):
    def __init__(self, path, img_shape):
        
        self.root = path
        self.width = img_shape[1]
        self.height = img_shape[0]
        self.images = os.listdir(path+'images\\')
        self.annotations = os.listdir(path+'annotations\\')
        self.length = len(self.images)
        
    def __len__(self):
        return self.length
    
    def __getitem__(self, idx):
        #print(self.root+'images\\'+self.images[idx])
        image = cv2.imread(self.root+'images\\'+self.images[idx])
        #print(image.shape)
        img_width = image.shape[1]
        img_height = image.shape[0]     #right?
        
        image  =cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        if ((self.width,self.height) != (-1,-1)):
            image = cv2.resize(image,(self.width,self.height),interpolation = cv2.INTER_AREA)
        else:
            self.width = img_width
            self.height = img_height
            
        image = image.transpose(2,0,1)
        image = (image/255.0)
        
        
        #bbox_left, bbox_top, bbox_width, bbox_height, category = map(float, data[:5])
        target = self.annotations[idx]    #*.txt file       
        
        boxes = []
        labels = []
        with open(self.root+'annotations\\'+target) as f:
            for line in f:
                data = line.split(',')#strip
                #print(list(map(float, data[:5])))
                box_left, box_top, box_width, box_height,smth, category = map(float, data[:6])
                
                box_width = 1 if box_width < 1 else box_width
                box_height = 1 if box_height < 1 else box_height
                
                box_right = box_left + box_width
                box_bottom = box_top + box_height
                #rescale box            
                box_left = (box_left/img_width) * self.width
                box_right = (box_right/img_width) * self.width
                box_top = (box_top/img_height) * self.height
                box_bottom = (box_bottom/img_height) * self.height
                boxes.append((box_left,box_top,box_right, box_bottom))
#                 for b in boxes:
#                     for bb in b:
#                         if bb > 480 or bb < 0:
#                             print(b,'\n',bb)
                labels.append(int(category))
                
        
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        #print(boxes)
        labels = torch.as_tensor(labels, dtype=torch.int64)
        target = {}
        target['boxes'] = boxes
        target['labels'] = labels
        target['image_id'] = torch.tensor((idx))
        
        return image, target # np.array, dict of lists

In [ ]:
train_dataset = VisDroneDataset(changed_dir+train_dir,(480,480))
val_dataset = VisDroneDataset(root_dir+val_dir,(-1,-1))

In [ ]:
def collate_fn(batch):
    images = []
    targets = []

    for img, target in batch:
        images.append(img)
        targets.append(target)

    return (images, targets)


In [ ]:
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True,collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=3, shuffle=True,collate_fn=collate_fn)

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
torch.hub.set_dir('D:\\torch_models')

In [ ]:
from torchvision.models import resnet50

In [ ]:
import torchvision
from torchvision.models.detection import FasterRCNN_ResNet50_FPN_Weights
from torchvision.models.detection import ssdlite320_mobilenet_v3_large

In [ ]:
def create_model_ssd():
    model = ssdlite320_mobilenet_v3_large(num_classes=12)
    print(model)
    return model

In [ ]:

def create_model_faster_rcnn(num_classes):
    # Load a pre-trained  R-CNN model 
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights="DEFAULT")
    for p in model.parameters():
        p.requires_grad=False
        
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    
    return model



In [ ]:
def load_best_model():
    with open('model.pkl','rb') as f:
        model = pickle.load(f)
        return  model
    

In [ ]:
model = create_model_faster_rcnn(12)

In [ ]:
model.train()


In [ ]:
import time
from tqdm import tqdm
def train_one_epoch(model, train_loader, optimizer):
    total_loss=0
    n=0
    prev_time = time.time()
    for data, target in tqdm(train_loader):
        
        optimizer.zero_grad()
        #print(len(data))
        data = torch.as_tensor(np.array(data),dtype=torch.float32).to(device)
        target = [{k: v.to(device) for k, v in t.items()} for t in target]
        
        #print(target)
        loss_dict = model(data, target)

        losses = sum(loss for loss in loss_dict.values())

        total_loss+=losses.item()
        losses.backward()
        optimizer.step()
        if(n % 100 == 0):
            print('loss', total_loss/(n+1))
            prev_time = time.time()
            
        
        n+=1
        del data, target
        #model = model.to(torch.device('cpu'))
        gc.collect()
        torch.cuda.empty_cache()
        
        #model = model.to(device)
    
    print(total_loss/n,'a')

In [ ]:
num_epoch = 50
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(params, lr=0.0005, weight_decay=0.000005)

lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

model = model.to(device)


In [ ]:

for epoch in range(num_epoch):
    train_one_epoch(model,train_loader,optimizer)
    lr_scheduler.step()

In [ ]:
#with open('model_trained_on_cropped.pkl', 'wb') as f:
#    pickle.dump(model, f)